
# Hierarchical Bayesian Linear Model with Experiment-Specific Offsets (TFP)

**Author:** _<Your Name>_  
**Course / PEC:** Hierarchical modeling with suspected zero-point offsets  
**Dataset:** `properMotions.csv` (columns: `lab, x, y, sigma`)

This notebook implements a hierarchical Bayesian linear model to infer the common linear relationship between \(x\) and \(y\) across \(N\) laboratories, and to test whether **experiment-dependent zero-point offsets** are supported by the data.

**What you'll find here:**
1. **Textual description** of the model, including **priors** and their **justification** (with references to *BDA3* and *Bayes Rules!*).  
2. **Model evaluation**: posterior predictive checks and PSIS-LOO model comparison.  
3. **Brief summary** template to report results once you run the notebook.



## 1. Model and priors (with justification)

We have observations \((x_{ij}, y_{ij})\) from lab \(i \in \{1,\ldots, N\}\) and measurement \(j \in \{1,\ldots, n_i\}\). The measurement noise is Gaussian with **known** lab-specific standard deviation \(\sigma_i\). Scientists suspect each lab may have a **zero-point calibration offset** \(o_i\). We compare two nested models.

### Model A (baseline, no offsets)
\[
y_{ij} \sim \mathcal{N}\!\big(m\,x_{ij} + b,\ \sigma_i\big).
\]

### Model B (hierarchical offsets; partial pooling)
\[
\begin{aligned}
y_{ij} &\sim \mathcal{N}\!\big(m\,x_{ij} + b + o_i,\ \sigma_i\big),\\
o_i \mid \tau &\sim \mathcal{N}(0,\ \tau),\quad \tau>0.
\end{aligned}
\]
Offsets \(o_i\) are **exchangeable** across labs, enabling **shrinkage** toward 0 when the data are weak—classic partial pooling (BDA3 **Ch. 5.1–5.7**).

### Standardization
For stable geometry with HMC/NUTS, we standardize:
\(
x^{\star}=(x-\bar x)/s_x,\quad y^{\star}=(y-\bar y)/s_y,\quad \sigma^{\star}=\sigma/s_y.
\)
We fit the model on the standardized scale and then transform back to original units.

### Priors (weakly informative, on standardized scale)
\[
m^{\star}\sim\mathcal{N}(0,1),\qquad
b^{\star}\sim\mathcal{N}(0,1),\qquad
\tau^{\star}\sim \text{Half-Normal}(1),\qquad
o_i^{\star} \mid \tau^{\star} \sim \mathcal{N}(0,\ \tau^{\star}).
\]
These encode that typical slopes/intercepts are within a few SDs a priori, and that offsets are centered at 0 with an unknown scale allowing the data to decide the extent of lab-specific bias. See BDA3 **Ch. 14–15** (regression & hierarchical linear models) and **Ch. 5.7** (hierarchical scale priors).

### Back-transformation to original units
\[
m = \frac{s_y}{s_x} m^{\star},\quad
b = \bar y + s_y\!\left(b^{\star} - m^{\star}\frac{\bar x}{s_x}\right),\quad
o_i = s_y\, o_i^{\star},\quad
\tau = s_y\, \tau^{\star}.
\]

### References for concepts
- **Likelihood × Prior → Posterior**: *Bayes Rules!* **Ch. 2** (posterior \(\\propto\) prior × likelihood).  
- **MCMC/HMC/NUTS approximation**: *Bayes Rules!* **Ch. 6**.  
- **Exchangeability & hierarchical models**: **BDA3 Ch. 5.1–5.7** (e.g., Eight Schools).  
- **Regression & hierarchical linear models**: **BDA3 Ch. 14** and **Ch. 15**.  
- **Posterior predictive checks**: **BDA3 Ch. 6**.  
- **Predictive performance & PSIS-LOO**: **BDA3 Ch. 7**.



## 2. Data loading and quick EDA


In [ ]:

# Standard scientific stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Load data (columns: lab, x, y, sigma)
DATA_PATH = Path('/mnt/data/properMotions.csv')  # adjust path if needed
df = pd.read_csv(DATA_PATH, header=None, names=['lab','x','y','sigma'])
df['lab'] = df['lab'].astype(int)

labs = np.sort(df['lab'].unique())
n_labs = labs.size
print(f"N rows = {len(df)}, N labs = {n_labs}")
print(df.head())

# Quick scatter: y vs x, colored by lab
fig, ax = plt.subplots(figsize=(6,4))
for lab in labs:
    d = df[df['lab']==lab]
    ax.scatter(d['x'], d['y'], s=20, alpha=0.7, label=f"lab {lab}")
ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_title("Observed data by lab")
ax.legend(ncols=3, fontsize=8)
plt.show()



### Standardization (for sampler geometry)


In [ ]:

x = df['x'].to_numpy().astype('float64')
y = df['y'].to_numpy().astype('float64')
sigma_obs = df['sigma'].to_numpy().astype('float64')
lab = df['lab'].to_numpy().astype(int)

x_mean, x_sd = x.mean(), x.std()
y_mean, y_sd = y.mean(), y.std()

x_star = (x - x_mean)/x_sd
y_star = (y - y_mean)/y_sd
sigma_star = sigma_obs / y_sd

# Map lab IDs to 0..n_labs-1
lab_to_idx = {lab_id:i for i, lab_id in enumerate(labs)}
lab_idx = np.array([lab_to_idx[v] for v in lab], dtype=int)

print("Standardized shapes:", x_star.shape, y_star.shape, sigma_star.shape)



## 3. Prior predictive simulation (to justify priors)

Simulate from the prior to verify that the implied data scale is reasonable. (BDA3 **Ch. 6**; useful for checking priors.)


In [ ]:

rng = np.random.default_rng(123)

def prior_predictive(num_draws=1000):
    m = rng.normal(0, 1, size=num_draws)
    b = rng.normal(0, 1, size=num_draws)
    tau = np.abs(rng.normal(0, 1, size=num_draws))  # Half-Normal(1)
    # Sample lab offsets
    o = rng.normal(0, tau[:, None], size=(num_draws, len(labs)))  # shape (draw, n_labs)
    # Generate y* at observed x* with known sigma*
    yrep = []
    for d in range(num_draws):
        mu = m[d]*x_star + b[d] + o[d, lab_idx]
        yrep.append(rng.normal(mu, sigma_star))
    return np.array(yrep)  # (draw, N)

# Example quick check (commented for speed):
# yrep = prior_predictive(200)
# fig, ax = plt.subplots(figsize=(6,4))
# ax.hist(yrep.ravel(), bins=40, density=True, alpha=0.7)
# ax.set_title("Prior predictive (y*), Model B")
# plt.show()



## 4. TFP implementation (NUTS) — Models A and B


In [ ]:

import tensorflow as tf
import tensorflow_probability as tfp

tfd, tfb, mcmc = tfp.distributions, tfp.bijectors, tfp.mcmc
tf.random.set_seed(42)

x_tf = tf.convert_to_tensor(x_star, dtype=tf.float64)
y_tf = tf.convert_to_tensor(y_star, dtype=tf.float64)
sigma_tf = tf.convert_to_tensor(sigma_star, dtype=tf.float64)
lab_idx_tf = tf.convert_to_tensor(lab_idx, dtype=tf.int32)
n_labs_tf = tf.constant(int(lab_idx.max()+1), dtype=tf.int32)



### 4.1 Model A — No offsets
\[
y^{\star}_n \sim \mathcal{N}(m^{\star} x^{\star}_n + b^{\star},\ \sigma^{\star}_n),\quad
m^{\star}, b^{\star} \sim \mathcal{N}(0,1).
\]


In [ ]:

def joint_no_offsets(x, sigma):
    def model():
        m = yield tfd.Normal(0., 1., name='m')
        b = yield tfd.Normal(0., 1., name='b')
        mu = m*x + b
        y = yield tfd.Independent(tfd.Normal(mu, sigma), 1, name='y')
    return tfd.JointDistributionCoroutineAutoBatched(model)

jd_A = joint_no_offsets(x_tf, sigma_tf)
jd_A_pinned = jd_A.experimental_pin(y=y_tf)

def tlp_A(m, b):
    return jd_A_pinned.log_prob(m, b)

init_A = [tf.constant(0., tf.float64), tf.constant(0., tf.float64)]
nuts_A = mcmc.NoUTurnSampler(target_log_prob_fn=tlp_A, step_size=0.2)
nuts_A = mcmc.SimpleStepSizeAdaptation(nuts_A, num_adaptation_steps=1000)

@tf.function(autograph=False, jit_compile=False)
def run_A(num_draws=2000, num_burnin=2000):
    return mcmc.sample_chain(num_results=num_draws,
                             num_burnin_steps=num_burnin,
                             current_state=init_A,
                             kernel=nuts_A,
                             trace_fn=lambda cs, kr: kr.inner_results.is_accepted)

# Example (leave commented unless you want to run):
# draws_A, acc_A = run_A(2000, 2000)
# m_A, b_A = draws_A



### 4.2 Model B — Hierarchical offsets
\[
\\begin{aligned}
y^{\\star}_n &\\sim \\mathcal{N}(m^{\\star} x^{\\star}_n + b^{\\star} + o^{\\star}_{\\ell(n)},\\ \\sigma^{\\star}_n),\\\\
o^{\\star}_i \\mid \\tau^{\\star} &\\sim \\mathcal{N}(0, \\tau^{\\star}),\\quad \\tau^{\\star}>0,\\\\
m^{\\star}, b^{\\star} &\\sim \\mathcal{N}(0,1),\\quad \\tau^{\\star} \\sim \\text{Half-Normal}(1).
\\end{aligned}
\]


In [ ]:

def joint_offsets(x, sigma, lab_idx, n_labs):
    def model():
        m = yield tfd.Normal(0., 1., name='m')
        b = yield tfd.Normal(0., 1., name='b')
        tau = yield tfd.HalfNormal(1., name='tau')
        o = yield tfd.Sample(tfd.Normal(0., tau), sample_shape=[n_labs], name='o')
        mu = m*x + b + tf.gather(o, lab_idx)
        y = yield tfd.Independent(tfd.Normal(mu, sigma), 1, name='y')
    return tfd.JointDistributionCoroutineAutoBatched(model)

jd_B = joint_offsets(x_tf, sigma_tf, lab_idx_tf, n_labs_tf)
jd_B_pinned = jd_B.experimental_pin(y=y_tf)

def tlp_B(m, b, tau, o):
    return jd_B_pinned.log_prob(m, b, tau, o)

init_B = [tf.constant(0., tf.float64),
          tf.constant(0., tf.float64),
          tf.constant(0.5, tf.float64),
          tf.zeros([int(n_labs_tf.numpy())], tf.float64)]

bij_B = [tfb.Identity(), tfb.Identity(), tfb.Softplus(), tfb.Identity()]

nuts_B = mcmc.NoUTurnSampler(target_log_prob_fn=tlp_B, step_size=0.2)
nuts_B = mcmc.TransformedTransitionKernel(nuts_B, bijector=bij_B)
nuts_B = mcmc.SimpleStepSizeAdaptation(nuts_B, num_adaptation_steps=1000)

@tf.function(autograph=False, jit_compile=False)
def run_B(num_draws=2000, num_burnin=2000):
    return mcmc.sample_chain(num_results=num_draws,
                             num_burnin_steps=num_burnin,
                             current_state=init_B,
                             kernel=nuts_B,
                             trace_fn=lambda cs, kr: kr.inner_results.is_accepted)

# Example (leave commented unless you want to run):
# draws_B, acc_B = run_B(2000, 2000)
# m_B, b_B, tau_B, o_B = draws_B



### 4.3 Back-transformation helpers


In [ ]:

def backtransform_mb(m_star, b_star, x_mean, x_sd, y_mean, y_sd):
    m = (y_sd/x_sd)*m_star
    b = y_mean + y_sd*(b_star - m_star*(x_mean/x_sd))
    return m, b

def backtransform_offsets(o_star, y_sd):
    return o_star * y_sd

def backtransform_tau(tau_star, y_sd):
    return tau_star * y_sd



## 5. Model evaluation

We use **posterior predictive checks** (graphical comparison of replicated vs. observed) and **PSIS-LOO** to compare Model A vs. Model B (BDA3 **Ch. 6–7**).



### 5.1 Posterior predictive checks (PPC)


In [ ]:

# Posterior predictive for Model B (as an example)
# import numpy as np
# import matplotlib.pyplot as plt

# def ppc_modelB(m_draws, b_draws, o_draws, n_show=50, seed=2024):
#     rng = np.random.default_rng(seed)
#     idx = rng.choice(m_draws.shape[0], size=min(n_show, m_draws.shape[0]), replace=False)
#     fig, ax = plt.subplots(figsize=(6,4))
#     ax.scatter(x_star, y_star, s=8, alpha=0.6, label='observed (stdzd)')
#     for k in idx:
#         mu = m_draws[k]*x_star + b_draws[k] + o_draws[k][lab_idx]
#         yrep = rng.normal(mu, sigma_star)
#         ax.plot(x_star, yrep, alpha=0.05)
#     ax.set_title("PPC — Model B on standardized scale"); ax.set_xlabel("x*"); ax.set_ylabel("y*")
#     plt.show()
#
# ppc_modelB(m_B.numpy(), b_B.numpy(), o_B.numpy())



### 5.2 PSIS-LOO model comparison


In [ ]:

# pip install arviz (if not available)
# import arviz as az
# import numpy as np
#
# def pointwise_ll_A(m_draws, b_draws):
#     mu = np.einsum('d,n->dn', m_draws, x_star) + b_draws[:, None]
#     return -0.5*np.log(2*np.pi*sigma_star**2) - 0.5*((y_star - mu)/sigma_star)**2
#
# def pointwise_ll_B(m_draws, b_draws, o_draws):
#     mu = (np.einsum('d,n->dn', m_draws, x_star) + b_draws[:, None] +
#           o_draws[:, lab_idx])
#     return -0.5*np.log(2*np.pi*sigma_star**2) - 0.5*((y_star - mu)/sigma_star)**2
#
# ll_A = pointwise_ll_A(m_A.numpy(), b_A.numpy())
# ll_B = pointwise_ll_B(m_B.numpy(), b_B.numpy(), o_B.numpy())
#
# id_A = az.from_dict(posterior={"m": m_A.numpy(), "b": b_A.numpy()},
#                     log_likelihood={"y": ll_A})
# id_B = az.from_dict(posterior={"m": m_B.numpy(), "b": b_B.numpy(), "o": o_B.numpy()},
#                     log_likelihood={"y": ll_B})
#
# loo_A = az.loo(id_A, pointwise=True)
# loo_B = az.loo(id_B, pointwise=True)
# comp = az.compare({"A_no_offsets": id_A, "B_offsets": id_B}, ic="loo", method="BB-pseudo-BMA")
# display(comp)
#
# print("\nInterpretation tip: If ΔELPD (B − A) >> SE, prefer Model B (offsets).")



## 6. Convergence diagnostics

Use \(\hat{R}\) and effective sample size (ESS); visualize traces (BDA3 **Ch. 10**).


In [ ]:

# import arviz as az
# id_B = az.from_dict(posterior={"m": m_B.numpy(), "b": b_B.numpy(), "tau": tau_B.numpy(), "o": o_B.numpy()})
# summary = az.summary(id_B, var_names=['m','b','tau','o'])
# display(summary)
# az.plot_trace(id_B, var_names=['m','b','tau'])
# plt.show()



## 7. Results (to fill after running)

- **Slope \(m\)** and **intercept \(b\)** in original units (means and 90%/95% credible intervals).  
- **Offsets \(o_i\)**: report posterior intervals; check which exclude 0.  
- **Hyper-scale \(\tau\)**: if concentrated near 0, offsets likely negligible; if away from 0, supports offsets.  
- **Model comparison**: report PSIS-LOO \(\Delta\)ELPD and SE; prefer the model with clearly higher ELPD.


In [ ]:

# Example transformation (after sampling):
# m_B_orig, b_B_orig = backtransform_mb(m_B.numpy(), b_B.numpy(), x_mean, x_sd, y_mean, y_sd)
# o_B_orig = backtransform_offsets(o_B.numpy(), y_sd)
# tau_B_orig = backtransform_tau(tau_B.numpy(), y_sd)
#
# # Example plot of offsets in original units
# def plot_offsets(o_draws_orig, labs, q=(0.05, 0.95)):
#     lo, hi = np.quantile(o_draws_orig, q, axis=0)
#     mu = o_draws_orig.mean(axis=0)
#     ix = np.arange(len(labs))
#     fig, ax = plt.subplots(figsize=(6,4))
#     ax.errorbar(ix, mu, yerr=[mu-lo, hi-mu], fmt='o', capsize=3)
#     ax.axhline(0, color='k', lw=1, ls='--')
#     ax.set_xticks(ix); ax.set_xticklabels([str(l) for l in labs])
#     ax.set_xlabel("lab"); ax.set_ylabel("offset $o_i$ (original units)"); ax.set_title("Posterior offsets (90% CI)")
#     plt.tight_layout(); plt.show()
#
# plot_offsets(o_B_orig, labs)



## 8. Brief summary (fill in after you run)
- Is there **evidence for experiment-dependent zero-point offsets**? (Use \(\tau\), \(o_i\) intervals, and PSIS-LOO.)  
- Final linear relationship \(y \approx m x + b\) (original units) with credible intervals.  
- Any labs with notably non-zero offsets?  
- Diagnostics and model-check highlights (e.g., PPC fit, Pareto-\(k\) warnings).



## References
- **BDA3 — Bayesian Data Analysis (3rd ed.)**
  - **Ch. 5.1–5.7**: Exchangeability & hierarchical models (e.g., Eight Schools).  
  - **Ch. 6**: Model checking & posterior predictive checks.  
  - **Ch. 7**: Predictive performance & PSIS-LOO.  
  - **Ch. 10**: Computation & diagnostics (\(\hat{R}\), ESS).  
  - **Ch. 14–15**: Regression & hierarchical linear models.
- **Bayes Rules!**
  - **Ch. 2**: Prior × likelihood \(\Rightarrow\) posterior.  
  - **Ch. 6**: MCMC for posterior approximation.  
  - **Ch. 15**: Hierarchical models.
